# Imports

In [1]:
import pandas as pd
import math

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# for file-management
import pickle as pkl
from pathlib import Path
import os.path
import sys

# Participant Infos

In [2]:
output = True
participants = list(range(1, 21))
# TODO remove
participants = list(range(1, 5))
smartphones = ["S3Mini", "S4", "N5X", "N6"]
event_time_diff = 3

sensors = [
    "acc",
    "gyro",
    "ori",
    "grav",
    "mag",
    "rot"
]

tasks = [
    "points",
    "fitts"
]

file_names = sensors + tasks

In [3]:
%%time
for PID in participants:
    for smartphone in smartphones:
        print("processing - participant:", str(PID), "smartphone:", smartphone)

        # set screen resolution
        if smartphone == "N5X":
            pixels = {"width": 1080, "height": 1920}
        elif smartphone == "S3Mini":
            pixels = {"width": 480, "height": 800}
        elif smartphone == "S4":
            pixels = {"width": 1080, "height": 1920}
        elif smartphone == "N6":
            pixels = {"width": 1440, "height": 2560}

        # Read Files
        raw_data = dict()

        if output: print("read files")
        for file in file_names:
            file_path = str(Path.home()) + "/data/raw/fapra_imu-" +  str(PID) + "-" + file + "-" + smartphone + "-0.csv"
            if not os.path.isfile(file_path):
                print(file_path + "not found")
                continue
            raw_data[file] = pd.read_csv(file_path, ";")

        # split by time in seperate lists
        if output: print("sort to specific interval")
        time_filtered_data = dict()
        for name in file_names:
            tmp = []
            for k, end in enumerate(raw_data["points"].time):
                start = int(raw_data["fitts"].time[k])
                # create mask for time interval
                mask = (raw_data[name]["time"] > start) & (raw_data[name]["time"] <= end)
                # only return items matching to mask
                tmp.append(raw_data[name].loc[mask])
            time_filtered_data[name] = tmp
        # list to dataframe
        time_filtered_data["points"] = pd.concat(time_filtered_data["points"])

        # scale screen
        time_filtered_data["points"]["x-press"] = time_filtered_data["points"]["x-press"].div(pixels["width"])
        time_filtered_data["points"]["x-circle"] = time_filtered_data["points"]["x-circle"].div(pixels["width"])
        time_filtered_data["points"]["y-press"] = time_filtered_data["points"]["y-press"].div(pixels["height"])
        time_filtered_data["points"]["y-circle"] = time_filtered_data["points"]["y-circle"].div(pixels["height"])

        # filter unique timestamps
        for k, item in enumerate(time_filtered_data["points"]):
            for sensor in sensors:
                time_filtered_data[sensor][k] = time_filtered_data[sensor][k].drop_duplicates(subset="time", keep="last")
        
        # Create Array
        result_interval = []
        if output: print("create-interval", end=' ')
        for k, point in time_filtered_data["points"].iterrows():
            if output: print(k, end=' ', flush=True)
            one_interval = []
            interval = dict()
            for sensor in sensors:
                interval[sensor] = time_filtered_data[sensor][k]

            # keeps index of each sensor
            position = dict()
            # keeps value of last sensor event
            last_values = dict()
            for sensor in sensors:
                position[sensor] = 0
                last_values[sensor] = 0

            # find maximum first timestamp in all sensors
            current_time = -1
            for sensor in sensors:
                if interval[sensor]["time"].iloc[position[sensor]] > current_time:
                    current_time = interval[sensor]["time"].iloc[position[sensor]]
                last_values[sensor] = interval[sensor][:].iloc[position[sensor]]
            
            # print("start", raw_data["fitts"].time.iloc[k])
            # print("first_data", current_time)
            # print("end", time_filtered_data["points"].iloc[k]["time"])

            # iterate as long as time has not reached max
            while current_time <= time_filtered_data["points"].iloc[k]["time"]:
                # iterate over alle sensors to find values before current_time (maybe one sensor is having multiple updates)
                while True:
                    all_valid = True
                    for sensor in sensors:
                        if position[sensor] + 1 >= len(interval[sensor]["time"]):
                            continue
                        if interval[sensor]["time"].iloc[position[sensor] + 1] <= current_time:
                            position[sensor] += 1
                            last_values[sensor] = interval[sensor][:].iloc[position[sensor]]
                            all_valid = False
                    if all_valid:
                        break
                one_interval.append(last_values.copy())
                current_time = current_time + event_time_diff
            result_interval.append(one_interval)
        
        # make numpy arrays (without dicts)
        if output: print()
        if output: print("make numpy-array interval", end=' ')
        final_intervals = []
        for k, interval in enumerate(result_interval):
            if output: print(k, end=' ', flush=True)
            current_interval = []
            for i in interval:
                event = []
                for sensor in i.values():
                    # except timestamp
                    for value in sensor[1:]:
                        event.append(value)
                current_interval.append(np.asarray(event))
            # make list to array, transpose and make 2d matrix
            final_intervals.append(np.array(np.asarray(current_interval).transpose()))
            
        if output: print()
        if output: print("make numpy-array points")
        final_points = time_filtered_data["points"][['x-press','x-circle','y-press','y-circle']].values
        final_result = [final_points, final_intervals]

        # save dump pickles
        data_path = str(Path.home()) + "/data/pickles/fapra_imu-processed-" +  str(PID) + "-" + smartphone + ".pkl"
        pkl.dump(final_result, open( data_path, "wb" ))

processing - participant: 1 smartphone: S3Mini
read files
sort to specific interval
create-interval 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 25

create-interval 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 27

create-interval 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 27

create-interval 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 27

create-interval 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 27

create-interval 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 27

create-interval 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 27

create-interval 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 27